In [1]:
# add path to sispeuede to sys.path in python
import sys
import pathlib
import warnings
warnings.filterwarnings("ignore")


path_git = pathlib.Path("/Users/usuario/git")
for subdir in [
    "sisepuede", 
    "sisepuede_data_pipeline",
    "sisepuede_juypyter"
]:
    _PATH_CUR = path_git.joinpath(subdir)
    if str(_PATH_CUR) not in sys.path:
        sys.path.append(str(_PATH_CUR))

path_pipeline = path_git.joinpath("sisepuede_data_pipeline")


import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import sisepuede.core.attribute_table as att
import sisepuede.core.support_classes as sc
import sisepuede.legacy.data_api as api
import sisepuede.manager.sisepuede_examples as sxl
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede.visualization.plots as svp


import time
from typing import *

# from sisepuede_data_pipeline
import lib.process_utilities as pu
import lib.sisepuede_data_constructs as dc
import lib._util as lutil

# from sisepuede_jupyter
import temp_update_fields_from_wv_to_main as temp 


# Setup SISEPUEDE elements

In [2]:
def get_file_structure(
    y0: int = 2015,
    y1: int = 2070,
) -> Tuple[sfs.SISEPUEDEFileStructure, att.AttributeTable]:
    """Get the SISEPUEDE File Structure and update the attribute table
        with new years.
    """
    # setup some SISEPUEDE variables and update time period
    file_struct = sfs.SISEPUEDEFileStructure(
        initialize_directories = False,
    )

    # get some keys
    key_time_period = file_struct.model_attributes.dim_time_period
    key_year = file_struct.model_attributes.field_dim_year


    ##  BUILD THE ATTRIBUTE AND UPDATE

    # setup the new attribute table
    years = np.arange(y0, y1 + 1, ).astype(int)
    attribute_time_period = att.AttributeTable(
        pd.DataFrame(
            {
                key_time_period: range(len(years)),
                key_year: years,
            }
        ),
        key_time_period,
        
    )

    # finally, update the ModelAttributes inside the file structure
    (
        file_struct
        .model_attributes
        .update_dimensional_attribute_table(
            attribute_time_period,
        )
    )

    # return the tuple
    out = (file_struct, attribute_time_period, )

    return out

    
# set up some paths
_PATH_CUR = pathlib.Path(os.getcwd())
_PATH_DATA = _PATH_CUR.joinpath("data")
_PATH_OUTPUT = _PATH_DATA.joinpath("output")

# export prefix
_PREFIX_FILENAME_DATASETBUILD_BY_REGION = "sisepuede_raw_inputs_latest_"


# model attributes and associated support classes
_EXAMPLES = sxl.SISEPUEDEExamples()
_FILE_STRUCTURE, _ATTRIBUTE_TABLE_TIME_PERIOD = get_file_structure()
matt = _FILE_STRUCTURE.model_attributes
regions = sc.Regions(matt, )
time_periods = sc.TimePeriods(matt, )

# setup models
models = sm.SISEPUEDEModels(
    matt,
    allow_electricity_run = True,
    fp_julia = _FILE_STRUCTURE.dir_jl,
    fp_nemomod_reference_files = _FILE_STRUCTURE.dir_ref_nemo,
    initialize_julia = True, 
)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   1587.6 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


In [21]:
##  SOME GLOBALS

df_example_input = _EXAMPLES("input_data_frame")
tab = regions.attributes.table


##  NOTE! Need to ignore these for the moment
# The GDP for these are incomplete:
#   - antigua_and_barbuda
#   - cayman_islands
# There are other issues with these countries:
#   - british_virgin_islands
#   - cuba
#   - curacao
#   - dominica
#   - grenada'
#   - saint_kitts_and_nevis
#   - saint_martin
#   - sint_maarten
#   - turks_and_caicos_islands
#   - united_states_virgin_islands
#
_REGIONS_TO_CHECK = [
    "antigua_and_barbuda", 
    "british_virgin_islands",
    "cayman_islands",
    "cuba",
    "curacao",
    "dominica",
    "grenada",
    "saint_kitts_and_nevis",
    "saint_martin",
    "sint_maarten",
    "turks_and_caicos_islands",
    "united_states_virgin_islands"
]

_REGIONS_BUILD = list(
    tab[
        tab["un_sub_region"].isin(["Latin America and the Caribbean"])
        & ~tab[regions.key].isin(_REGIONS_TO_CHECK)    
    ][regions.key]
    .unique()
)
_REGIONS_BUILD = sorted(
    _REGIONS_BUILD +
    [
        "bulgaria",
        "egypt",
        "libya",
        "morocco",
        "united_republic_of_tanzania"
    ]
)


_REGIONS_ISO = [
    regions.return_region_or_iso(x, return_type = "iso", )
    for x in _REGIONS_BUILD
]


# Setup old repository and read data

In [20]:
repo_old = api.SISEPUEDEBatchDataRepository(
    "/Users/usuario/git/sisepuede_data", 
    matt,
)

print("Getting old repository data...")
df_old = repo_old.read(None)
print("Old repository data complete.")

Getting old repository data...
Old repository data complete.


# Setup new repository

In [60]:
importlib.reload(dc.sc)
importlib.reload(dc.sfs.ma)
importlib.reload(dc.sfs)
importlib.reload(dc)
construct = dc.SISEPUEDEDataConstructs(
    path_output_database = "/Users/usuario/git/sisepuede_data_pipeline/sisepuede_inputs.sqlite",
)

path_repo = pathlib.Path("/Users/usuario/SISEPUEDE_DATA_REPOSITORY")
repo = pu.Repository(
    {
        "local": {
            "path": str(path_repo)
        }
    }
)

# get from pipeline
df_from_pipeline = construct.build_inputs_from_database(
    regions_keep = _REGIONS_BUILD,
    join = "outer"
)


In [75]:

def function_combine(
    df_repo_new: pd.DataFrame,
    df_repo_old: pd.DataFrame,
    df_example: pd.DataFrame,
    region_iso: str,
    dropnas_from_old: bool = True,
    years: Union[List[int], None] = None,
) -> pd.DataFrame:
    """Combine DataFrames by region, combining in a hierarchy
    """
    
    ##  FORMAT A BASE 

    # start by setting years
    if not sf.islistlike(years):
        years = time_periods.all_years

    
    df_base = (
        df_repo_old[
            df_repo_old[repo_old.field_repo_iso].isin([region_iso])
            & (df_repo_old[time_periods.field_year] >= min(years))
        ]
        .copy()
        .rename(
            columns = {
                repo_old.field_repo_iso: regions.field_iso,
                repo_old.field_repo_year: time_periods.field_year,
            }
        )
    )

    # QUICK FIX!
    #if "pop_lvst_initial_chickens" in df_base.columns:
    #    df_base["pop_lvst_initial_chickens"] *= 1000
    #    print("NOTE! Adjusting poultry population")

    # quick drop
    if dropnas_from_old:
        cols_drop = [
            x for x in df_base.columns 
            if df_base[x].dropna().shape[0] == 0
        ]
        df_base = df_base.drop(columns = cols_drop, )
        
    
    df_base[time_periods.field_year] = df_base[time_periods.field_year].astype(int)    
    df_base = (
        pd.merge(
            pd.DataFrame({time_periods.field_year: years, }),
            df_base,
            how = "left",
        )
        .ffill()
        .bfill()
    )


    ##  ADD IN PIPELINE DATA

    df_out = sf.match_df_to_target_df(
        df_base, 
        df_from_pipeline,
        [
            construct.time_periods.field_year,
            regions.field_iso,
        ],
        overwrite_only = False,
    )
    
    df_out[time_periods.field_year] = df_out[time_periods.field_year].astype(int)
    df_out = (time_periods.years_to_tps(df_out, ))


    ##  PULL MISSING FIELDS FROM EXAMPLE DF
    
    # fields not in base
    fields_missing = [
        x for x in df_example.columns 
        if (x not in df_out.columns) 
        and (x in matt.all_variable_fields_input)
    ]
    
    # specify fields to pull from the example
    fields_from_ex = [
        x for x in fields_missing 
        if not (
            False#x.startswith("frac_lndu_")
            #x.startswith("factor_lndu")
            #or x.startswith("frac_lndu_")
        )
    ]
    
    fields_from_ex = [
        x for x in fields_from_ex
        if (x not in df_out.columns)
    ]

    # merge in from ex
    df_out = (
        pd.merge(
            df_out,
            df_example[fields_from_ex + [time_periods.field_time_period]],
            how = "left"
        )
        .ffill()
        .bfill()
    )


    ##  TEMPORARY SCRIPT FOR MOVING FROM working_version TO latest full version
    
    df_out = temp.update_fields(
        df_out,
        matt,
    )

    return df_out


df_mex = function_combine(
    df_from_pipeline,
    df_old,
    df_example_input,
    "MEX",
)



# Run the next cell to build a composite file and/or export individual fiels (set `export = True` to do this)

In [72]:

df_out = []
export = True
# years = range(2015, 2071)

def build_path_for_region_file(
    iso: str,
) -> pathlib.Path:
    """Build the output path for a file
    """
    fn = f"{_PREFIX_FILENAME_DATASETBUILD_BY_REGION}{iso}.csv"
    path_out = _PATH_OUTPUT.joinpath(fn)
    
    return path_out



for iso in _REGIONS_ISO:

    df_cur = function_combine(
        df_from_pipeline,
        df_old,
        df_example_input,
        iso,
    )    
    df_cur[regions.field_iso] = iso

    # write to the output location?
    if export:
        path_write = build_path_for_region_file(iso, )
        df_cur.to_csv(
            path_write,
            encoding = "UTF-8",
            index = None,
        )
    
    df_out.append(df_cur,)
    print(f"Region {iso} complete")

df_out = sf._concat_df(df_out, )




Region ARG complete
Region ABW complete
Region BHS complete
Region BRB complete
Region BLZ complete
Region BOL complete
Region BRA complete
Region BGR complete
Region CHL complete
Region COL complete
Region CRI complete
Region DOM complete
Region ECU complete
Region EGY complete
Region SLV complete
Region GTM complete
Region GUY complete
Region HTI complete
Region HND complete
Region JAM complete
Region LBY complete
Region MEX complete
Region MAR complete
Region NIC complete
Region PAN complete
Region PRY complete
Region PER complete
Region PRI complete
Region LCA complete
Region VCT complete
Region SUR complete
Region TTO complete
Region TZA complete
Region URY complete
Region VEN complete


In [74]:
df_mex[[x for x in df_tmp.columns if "pop_lvst" in x]]

,pop_lvst_initial_buffalo,pop_lvst_initial_cattle_dairy,pop_lvst_initial_cattle_nondairy,pop_lvst_initial_chickens,pop_lvst_initial_goats,pop_lvst_initial_horses,pop_lvst_initial_mules,pop_lvst_initial_pigs,pop_lvst_initial_sheep
0,0.0,1722276.0,1568774.0,200425000.0,6231386.0,219500.0,950000.0,7979.0,18509601.0
1,0.0,1727301.0,1572699.0,208799000.0,5600000.0,238000.0,932000.0,7948.0,19870000.0
2,0.0,1757069.0,1606931.0,214049000.0,5205000.0,247000.0,930000.0,7909.0,19863000.0
3,0.0,1802839.0,1638161.0,219202000.0,5731000.0,249830.0,934000.0,7866.0,19880000.0
4,0.0,1751410.0,1576590.0,224773000.0,5993000.0,252074.0,927000.0,7869.0,21591000.0
5,0.0,1779230.0,1387670.0,227202000.0,5960600.0,252056.0,925000.0,7909.0,22088800.0
6,0.0,1776120.0,1402756.0,231483000.0,6207391.0,236296.0,891440.0,7902.0,22726481.0
7,0.0,1696204.0,1404705.0,235764000.0,6045200.0,237235.0,891440.0,7895.0,21800556.0
8,0.0,1696204.0,1404705.0,235764000.0,6045200.0,237235.0,891440.0,7895.0,21800556.0
9,0.0,1696204.0,1404705.0,235764000.0,6045200.0,237235.0,891440.0,7895.0,21800556.0


# option to test


In [73]:
region_run = "MEX"
df_run = models.project(
    df_out[
        df_out[regions.field_iso].isin([region_run])
    ].reset_index(drop = True),
    time_periods_base = np.arange(12),
    verbose = True,
)



2025-23-Oct 10:17:19.472 Opened SQLite database at /Users/usuario/git/sisepuede_region_nbs/generic_regions/tmpwqdk5uk2.sqlite.
2025-23-Oct 10:17:19.644 Added NEMO structure to SQLite database at /Users/usuario/git/sisepuede_region_nbs/generic_regions/tmpwqdk5uk2.sqlite.
2025-23-Oct 10:17:38.607 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.
2025-23-Oct 10:17:38.608 Validated run-time arguments.
2025-23-Oct 10:17:38.608 Connected to scenario database. Path = /Users/usuario/git/sisepuede_region_nbs/generic_regions/tmpwqdk5uk2.sqlite.
2025-23-Oct 10:17:38.973 Dropped pre-existing result tables from database.
2025-23-Oct 10:17:39.183 Created parameter views and indices.
2025-23-Oct 10:17:39.190 Created temporary tables.
2025-23-Oct 10:17:39.201 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-23-Oct 10:17:39.202 Verified that transmission modeling is not enabled.
2025-23-Oct 10:17:39.408 Defined dimensions.

IOStream.flush timed out


2025-23-Oct 10:18:09.570 Queued constraint E2b_AnnualEmissionProduction for creation.
2025-23-Oct 10:18:09.895 Queued constraint E4_EmissionsPenaltyByTechnology for creation.
2025-23-Oct 10:18:09.915 Queued constraint E5_DiscountedEmissionsPenaltyByTechnology for creation.
2025-23-Oct 10:18:09.962 Queued constraint E6_EmissionsAccounting1 for creation.
2025-23-Oct 10:18:10.326 Queued constraint E7_EmissionsAccounting2 for creation.
2025-23-Oct 10:18:10.431 Queued constraint E8_AnnualEmissionsLimit for creation.
2025-23-Oct 10:18:10.435 Queued constraint E9_ModelPeriodEmissionsLimit for creation.
2025-23-Oct 10:18:10.435 Queued 97 standard constraints for creation.
2025-23-Oct 10:18:10.435 Finished scheduled task to add constraints to model.
2025-23-Oct 10:18:10.435 Added 97 standard constraints to model.
2025-23-Oct 10:18:10.436 Defined model objective.
2025-23-Oct 10:33:12.959 Solved model. Solver status = INFEASIBLE.
2025-23-Oct 10:33:12.962 Solver did not find a solution for model. 

In [85]:
df_examples[[x for x in df_examples.columns if x.startswith("frac_wali_ww_domestic_rural_")]]

,frac_wali_ww_domestic_rural_treatment_path_advanced_aerobic,frac_wali_ww_domestic_rural_treatment_path_advanced_anaerobic,frac_wali_ww_domestic_rural_treatment_path_latrine_improved,frac_wali_ww_domestic_rural_treatment_path_primary,frac_wali_ww_domestic_rural_treatment_path_secondary_aerobic,frac_wali_ww_domestic_rural_treatment_path_secondary_anaerobic,frac_wali_ww_domestic_rural_treatment_path_septic,frac_wali_ww_domestic_rural_treatment_path_latrine_unimproved,frac_wali_ww_domestic_rural_treatment_path_untreated_no_sewerage,frac_wali_ww_domestic_rural_treatment_path_untreated_with_sewerage
0,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
1,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
2,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
3,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
4,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
5,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
6,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
7,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
8,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909
9,0.0,0.0,0.001176,0.278209,0.0,0.0,0.461079,0.001176,0.227451,0.030909


In [82]:
df_old[
    ~df_old["ef_lndu_conv_croplands_to_croplands_gg_co2_ha"].isna()
    &df_old["iso_code3"].isin(["EGY"])
]["frac_wali_ww_domestic_rural_treatment_path_advanced_anaerobic"]

7608   NaN
7609   NaN
7610   NaN
7611   NaN
7612   NaN
7613   NaN
7614   NaN
7615   NaN
7616   NaN
7617   NaN
7618   NaN
7619   NaN
7620   NaN
7621   NaN
7622   NaN
7623   NaN
7624   NaN
7625   NaN
7626   NaN
7627   NaN
7628   NaN
7629   NaN
7630   NaN
7631   NaN
7632   NaN
7633   NaN
7634   NaN
7635   NaN
7636   NaN
7637   NaN
7638   NaN
7639   NaN
7640   NaN
7641   NaN
7642   NaN
7643   NaN
Name: frac_wali_ww_domestic_rural_treatment_path_advanced_anaerobic, dtype: float64